<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/13-rnn-lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lotto

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
url = 'https://raw.githubusercontent.com/dantae74/machine-learning/main/dataset/lotto.csv'
df = pd.read_csv(url)

In [9]:
df.head()

,seq,date,num,won,1,2,3,4,5,6
0,964,2021.5.22,10,"2,345,861,063",6,21,36,38,39,43
1,963,2021.5.15,15,"1,476,478,125",6,12,19,23,34,42
2,962,2021.5.8,12,"1,940,906,094",1,18,28,31,34,43
3,961,2021.5.1,9,"2,575,231,209",11,20,29,31,33,42
4,960,2021.4.24,10,"2,401,133,213",2,18,24,30,32,45
